# Evaluation metrics

> Different metrics used for evaluation.

In [ ]:
#| default_exp inference.eval_metrics

In [ ]:
#| export
from genQC.imports import *
from scipy.stats import unitary_group

## Base norm

In [ ]:
#| export
class BaseNorm(abc.ABC): 
    """Base class for norms."""
    
    @staticmethod
    @abc.abstractmethod
    def distance(approx_U: torch.Tensor, target_U: torch.Tensor) -> torch.Tensor: raise NotImplementedError()
    
    @staticmethod
    @abc.abstractmethod
    def name() -> str: raise NotImplementedError()

## Unitary distances

In [ ]:
#| export
class UnitaryFrobeniusNorm(BaseNorm):
    """
    The Frobenius-Norm for unitaries: defined in https://arxiv.org/pdf/2106.05649.pdf.
    """

    def __call__(self, approx_U: torch.Tensor, target_U: torch.Tensor) -> torch.Tensor:        
        return Unitary_FrobeniusNorm.distance(approx_U, target_U)
    
    @staticmethod
    def distance(approx_U: torch.Tensor, target_U: torch.Tensor) -> torch.Tensor:
        d = 0.5 * torch.linalg.matrix_norm((approx_U-target_U), ord="fro")**2
        return d
        
    @staticmethod
    def name() -> str: return "Frobenius-Norm"

In [ ]:
#| export
class UnitaryInfidelityNorm(BaseNorm):
    """
    The Infidelity-Norm for unitaries: defined in https://link.aps.org/accepted/10.1103/PhysRevA.95.042318, TABLE I: 1.
    """

    def __call__(self, approx_U: torch.Tensor, target_U: torch.Tensor) -> torch.Tensor:        
        return Unitary_infidelity.distance(approx_U, target_U)
    
    @staticmethod
    def distance(approx_U: torch.Tensor, target_U: torch.Tensor) -> torch.Tensor: 
        """Supports batched intputs, can be used as loss. Input shapes [b, n, n] or [n, n]."""
        d = torch.matmul(torch.transpose(target_U, -2, -1).conj(), approx_U) # out [b, n, n] or [n, n]
        d = torch.diagonal(d, offset=0, dim1=-2, dim2=-1).sum(-1)  # do partial (batched) trace, out [b, n] or [n]      
        d = 1.0 - (d / target_U.shape[-1]).abs().square()
        return d
        
    @staticmethod
    def name() -> str: return "Unitary-Infidelity"

Test the metrics on random unitaries:

In [ ]:
approx_U = torch.tensor(unitary_group.rvs(8))
target_U = torch.tensor(unitary_group.rvs(8))

In [ ]:
print(UnitaryFrobeniusNorm.name())
UnitaryFrobeniusNorm.distance(target_U, target_U), UnitaryFrobeniusNorm.distance(approx_U, target_U)

Frobenius-Norm


(tensor(0., dtype=torch.float64), tensor(8.5523, dtype=torch.float64))

In [ ]:
print(UnitaryInfidelityNorm.name())
UnitaryInfidelityNorm.distance(target_U, target_U), UnitaryInfidelityNorm.distance(approx_U, target_U)

Unitary-Infidelity


(tensor(4.4409e-16, dtype=torch.float64), tensor(0.9895, dtype=torch.float64))

# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()